In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory

import tensorflow as tf
import tensorflow_datasets as tfds

Defining image shape

In [3]:
img_width = 512
img_height = 384
colors = 3

Connecting Google Collab with Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Creating datasets

In [28]:
ds_train = image_dataset_from_directory(
    '/content/gdrive/My Drive/trashnet/dataset_project/dataset_train',
    labels='inferred',
    label_mode = 'categorical',
    batch_size=16,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training',
)

Found 2163 files belonging to 6 classes.
Using 1515 files for training.


In [ ]:
for image, label in ds_train.take(2):
  print(image)
  print(label)

In [29]:
ds_val = image_dataset_from_directory(
    '/content/gdrive/My Drive/trashnet/dataset_project/dataset_train',
    labels='inferred',
    label_mode = 'categorical',
    batch_size=16,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='validation',
)

Found 2163 files belonging to 6 classes.
Using 648 files for validation.


In [30]:
ds_test = image_dataset_from_directory(
    '/content/gdrive/My Drive/trashnet/dataset_project/dataset_test',
    labels='inferred',
    label_mode = 'categorical',
    batch_size=16,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
)

Found 364 files belonging to 6 classes.


Preprocessing datasets

In [31]:
def preprocess(image,label):
  image = image / 255.
  return image, label

In [32]:
ds_train = ds_train.map(preprocess)

In [33]:
ds_val = ds_val.map(preprocess)

In [34]:
ds_test = ds_test.map(preprocess)

Defining CNN model

In [35]:
def initialize_model():
  
  model = Sequential()

  model.add(layers.Conv2D(16, (4,4), input_shape=(img_height, img_width, colors), activation='relu'))
  model.add(layers.Conv2D(32, (3,3), activation='relu'))

  model.add(layers.Flatten())

  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dense(6, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

Initializing and fitting model

In [36]:
model = initialize_model()

In [37]:
es = EarlyStopping(patience=15, restore_best_weights=True)

In [38]:
model.fit(ds_train, validation_data = ds_val, epochs=1000, callbacks=[es])

Epoch 1/1000
95/95 [==============================] - 22s 221ms/step - loss: 11.0174 - accuracy: 0.3129 - val_loss: 1.6588 - val_accuracy: 0.3179
Epoch 2/1000
95/95 [==============================] - 20s 210ms/step - loss: 1.1778 - accuracy: 0.5452 - val_loss: 1.6260 - val_accuracy: 0.4244
Epoch 3/1000
95/95 [==============================] - 20s 210ms/step - loss: 0.7020 - accuracy: 0.7888 - val_loss: 2.2388 - val_accuracy: 0.4151
Epoch 4/1000
95/95 [==============================] - 20s 208ms/step - loss: 0.2848 - accuracy: 0.9426 - val_loss: 2.1727 - val_accuracy: 0.4228
Epoch 5/1000
95/95 [==============================] - 20s 208ms/step - loss: 0.2280 - accuracy: 0.9538 - val_loss: 2.5509 - val_accuracy: 0.4444
Epoch 6/1000
95/95 [==============================] - 20s 210ms/step - loss: 0.1905 - accuracy: 0.9670 - val_loss: 3.4829 - val_accuracy: 0.3827
Epoch 7/1000
95/95 [==============================] - 20s 211ms/step - loss: 0.1594 - accuracy: 0.9657 - val_loss: 3.6937 - val_a

Evaluating model

In [39]:
model.evaluate(ds_test)

23/23 [==============================] - 2s 65ms/step - loss: 1.4795 - accuracy: 0.4643


[1.479503870010376, 0.4642857015132904]